# mdx VMのデプロイとVCP既存サーバ (SSH) モード セットアップ

このNotebookでは、mdx仮想マシンのデプロイと、VCノードとして利用するためのセットアップを行います。

## Notebook実行環境の準備

1. このNotebookを実行するために必要なコマンドのパッケージをJupyter Notebookサーバに追加
2. mdx REST API操作用の [mdx REST Client for Python](https://github.com/nii-gakunin-cloud/mdx-rest-client-python/) のインストール
3. mdx VMにSSHログインするためのキーペア作成
4. mdx のREST APIエンドポイントに接続できることの確認

### mdx REST API操作ライブラリのインストール

- [mdx REST Client for Python](https://github.com/nii-gakunin-cloud/mdx-rest-client-python/) 

In [ ]:
!git clone https://github.com/nii-gakunin-cloud/mdx-rest-client-python.git

In [ ]:
# mdx_{ext,lib}.py ファイルのパスを取得
import os
import sys
mdx_lib_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), './mdx-rest-client-python/src')

# モジュール検索パスを追加
sys.path.append(mdx_lib_path)

from mdx_ext import MdxResourceExt

### mdx VMにSSHログインするためのキーペア作成

- mdx VMにSSHログインするためのキーペアを作成します。
  - 公開鍵は、VMのデプロイパラメータとして使用

In [ ]:
ssh_private_key_path = '~/.ssh/id_rsa'
ssh_public_key_path = f'{ssh_private_key_path}.pub'

In [ ]:
!mkdir -p -m 700 $(dirname {ssh_private_key_path})
!test -f {ssh_private_key_path} || ssh-keygen -t rsa -f {ssh_private_key_path} -N ""

### mdx REST API エンドポイント接続確認

- HTTPステータスコード 200 が返ることを確認します。

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておく。
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## mdxでのVMデプロイ準備

1. mdx REST API 認証トークン設定
2. mdxの対象プロジェクトの確認と設定
3. mdxの対象ネットワークの確認と設定
4. VMパラメータ設定

### mdx REST API 認証トークンの設定

In [ ]:
from getpass import getpass
mdx_token = getpass()

In [ ]:
mdx = MdxResourceExt(mdx_token)

### mdxの対象プロジェクトの確認と設定

- 利用可能なmdxのプロジェクト情報を確認し、操作対象を設定します。

In [ ]:
import json
projects = mdx.get_assigned_projects()
for org in projects:
    print(json.dumps(org["projects"], indent=2, ensure_ascii=False))

以降の操作対象とする「プロジェクト名 `name`」 を設定してください。

In [ ]:
mdx_project_name = 

In [ ]:
mdx.set_current_project_by_name(mdx_project_name)

### mdxの対象ネットワークの確認と設定

- プロジェクトで利用可能なネットワークセグメント情報を取得し、操作対象を設定します。

In [ ]:
segments = mdx.get_segments()
print(json.dumps(segments, indent=2, ensure_ascii=False))

操作対象を配列要素のindex値で指定してください。（単一の場合は先頭要素 0 番目を指定）

In [ ]:
mdx_segment_id = mdx.get_segments()[0]["uuid"]
print(mdx_segment_id)

### VMパラメータ設定

VMへのsshログインで使用する公開鍵を読み込みます。

In [ ]:
import os
with open(os.path.expanduser(ssh_public_key_path)) as f:
    ssh_shared_key = f.read()
print(ssh_shared_key)

デプロイするVMのパラメータを設定します。（参考: [ハードウェアのカスタマイズ項目](https://docs.mdx.jp/ja/main/create_vm.html#deploy-settings) )

- CPUパック数
- ディスクサイズ (GB)
- VM名 **（プロジェクト内で重複しないように注意）**

In [ ]:
cpu_pack_num = 3
disk_size_gb = 40
vm_name = 

In [ ]:
# 推奨版、東京大学制作、20231221-1010版
DEFAULT_TEMPLATE_NAME = "UT-20231221-1010-ubuntu-2004-server"
DEFAULT_CATALOG = "16a41081-a1cf-428e-90d0-a147b3aa6fc2"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=cpu_pack_num,
    pack_type="cpu",
    disk_size=disk_size_gb,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=mdx_segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
    service_level="spot",
)

## VMデプロイ

mdx VMにIPアドレスが設定されるまで5分程度を要します。

In [ ]:
mdx.deploy_vm(vm_name, mdx_spec.copy(), wait_for=False)
print(f"{vm_name} deployed.")

In [ ]:
from mdx_ext import SLEEP_TIME_SEC, DEPLOY_VM_SLEEP_COUNT
import time
import re

start_time = time.time()
for _ in range(DEPLOY_VM_SLEEP_COUNT):
    info = mdx.get_vm_info(vm_name)
    if info["status"] == "PowerON":
        addr = info.get("service_networks", [{}])[0].get("ipv4_address", [None])[0]
        if re.match(r"^[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+$", addr):
            print(f"{vm_name} is now up with address {addr}.")
            break
    elapsed = int(time.time() - start_time)
    print(f"Waiting for {vm_name} to be up... (elapsed time: {elapsed} sec)")
    time.sleep(SLEEP_TIME_SEC)

VMに付与されたプライベートIPv4アドレス (先頭のアドレス) を取得します。

In [ ]:
host_ip_address = info["service_networks"][0]["ipv4_address"][0]
host_ip_address

## SSH初回ログイン時のパスワード設定

- 仮想マシンテンプレート「推奨版、東京大学制作、20220412-2043版」では、
  SSHでの初回ログイン時にOSのパスワード設定が求められる。
- ログインユーザ名は `mdxuser`
- 秘密鍵は、VMデプロイ時に指定した公開鍵 `ssh_shared_key` に対応する鍵ファイルを使用

In [ ]:
# sshログインユーザ名
ssh_user_name = 'mdxuser'

# 設定するOSパスワード
mdx_user_password = 'vmPass01'

expectコマンドのPython版である pexpect ライブラリを用いてOSのパスワードを設定します。

In [ ]:
import pexpect

user = ssh_user_name
host = host_ip_address
key = ssh_private_key_path
password = mdx_user_password

command = f"ssh -oStrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i {key} {user}@{host}"

child = pexpect.spawn(command, encoding='utf-8', timeout=30)
child.expect("New password: ")
child.sendline(password)
child.expect("Retype new password: ")
child.sendline(password)
child.wait()

マシンのホスト名に仮想マシン名を設定します。

In [ ]:
command = f"ssh -oStrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i {key} {user}@{host} sudo hostnamectl set-hostname %s" % (vm_name)
pexpect.run(command)


マシンのIPアドレスを固定します。

In [ ]:
import os
scripts_dir = os.path.join(os.path.dirname(os.path.join(os.getcwd())), 'scripts')
!ssh -oStrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i {key} {user}@{host} 'sudo bash -s ' < {scripts_dir}/mdx_change_addr.sh {host_ip_address}

## vcpsdk利用準備

vcpsdkを利用してVCノードの設定を行うため、vcpsdkの公開鍵をノードに登録します。  
以下のセルを実行し、vcpsdkを初期化します。

In [ ]:
# VCP REST API アクセストークンの入力
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
# VCP SDK 初期化
from vcpsdk.vcpsdk import VcpSDK
sdk = VcpSDK(vcc_access_token)

公開鍵をノードのauthorized_keysに登録します。

In [ ]:
command = f"ssh -oStrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -i {key} {user}@{host} echo %s >> ~/.ssh/authorized_keys" % (sdk.get_publickey())
pexpect.run(command)

## Dockerのインストール

- docker-ceのインストールスクリプトをmdx VM上で実行します。

In [ ]:
# docker-ce インストールスクリプト
install_script = '''#!/bin/sh
sudo apt-get -qq update
sudo apt-get -qq install -y ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu \
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get -qq update
sudo apt-get -qq install -y docker-ce
echo 'Port 20022' | sudo tee -a /etc/ssh/sshd_config
sudo systemctl restart sshd
'''

install_sh_file = '/tmp/install_docker.sh'
with open(install_sh_file, 'w') as f:
    f.write(install_script)

!ssh -oStrictHostKeyChecking=no -i {ssh_private_key_path} {ssh_user_name}@{host} 'sh' < {install_sh_file}

Dockerがインストールできたことを確認します。

In [ ]:
!ssh -i {ssh_private_key_path} -p 20022 {ssh_user_name}@{host} sudo docker version

## (付録) mdx VMの各種操作

### 仮想マシンの状態取得

In [ ]:
# 仮想マシンの状態取得
vm_info = mdx.get_vm_info(vm_name)
print(json.dumps(info, indent=2, ensure_ascii=False))

### プロジェクトの仮想マシンのリスト取得

In [ ]:
# プロジェクトの仮想マシンのリスト取得
lst = mdx.get_vm_list()
print(json.dumps(lst, indent=2, ensure_ascii=False))

### 仮想マシンのシャットダウン、削除

In [ ]:
# 仮想マシンのシャットダウン
mdx.power_shutdown_vm(vm_name, wait_for=True)

In [ ]:
# 仮想マシンの削除
mdx.destroy_vm(vm_name, wait_for=True)

### 仮想マシンテンプレートカタログの取得

In [ ]:
# 仮想マシンテンプレートのリスト取得
catalogs = mdx.get_vm_catalogs()
print(json.dumps(catalogs["results"], indent=2, ensure_ascii=False))